In [1]:
import mesa
import numpy as np
from random import sample
from random import choices
from random import randrange
import random
import pandas as pd
import datetime

In [2]:
df_edges = pd.DataFrame(columns = ["ID_1", "ID_2", "TS_start", "TS_end", "Type", "Edge_ID"])
df_nodes = pd.DataFrame(columns = ["ID", "State", "Datetime"])
df_properties = pd.DataFrame(columns = ["ID", "Sex", "Age"])

num_agents = 10

In [3]:
# Array utili
encounters_prob = {"wave": 0.3, "talk": 0.2, "kiss": 0.05, "lunch": 0.025, "dinner": 0.025, "party": 0.05, "lecture": 0.25, "tango_lesson": 0.1}
edge_id = 0
timestamp_start_array = []

In [4]:
# Functions
def check_times(df, agent_id, timestamp_start, timestamp_end):
    global nearest_ts_start_row, df_agent_sorted
    """This function returns if a certain encounter can be produced
    
    Parameters:
    
        - df: the dataframe containing previous encounters
        - agent_id
        - timestamps start and end of the newly generated encounter
    
    """
    df_agent = df.loc[df["ID_1"]==agent_id,:]
    df_agent_sorted = df_agent.sort_values("TS_start")
    ts_starts = df_agent_sorted["TS_start"]
    nearest_ts_start_row = [i for (i, ts) in enumerate(ts_starts) if timestamp_start > ts][-1]
    if timestamp_start > df_agent_sorted.iloc[nearest_ts_start_row]["TS_end"]:
        ts_ends = df_agent_sorted["TS_end"]
        nearest_ts_end_row = [i for (i, ts) in enumerate(ts_ends) if timestamp_end < ts]
        if bool(nearest_ts_end_row)==False:
            return True
        elif timestamp_end < df_agent_sorted.iloc[nearest_ts_end_row[0]]["TS_start"]:
            return True
    return False       
         
def find_overlaps(df_times_encounter, df_edges, edge_id, enc_type):
    global new_row_1, new_row_2, time_starts, time_ends
    for i in range(len(df_times_encounter.index)-1):
        for j in range(i+1, len(df_times_encounter.index)):
            time_starts = [df_times_encounter.iloc[i]["TS_start"], df_times_encounter.iloc[j]["TS_start"]]
            time_ends = [df_times_encounter.iloc[i]["TS_end"], df_times_encounter.iloc[j]["TS_end"]]
            if max(time_starts) < min(time_ends):
                new_row_1 = {"ID_1": df_times_encounter.iloc[i]["ID"], "ID_2": df_times_encounter.iloc[j]["ID"], "TS_start": max(time_starts), "TS_end": min(time_ends), "Type": enc_type, "Edge_ID": edge_id}
                df_edges.loc[len(df_edges)] = new_row_1
                new_row_2 = {"ID_1": df_times_encounter.iloc[j]["ID"], "ID_2": df_times_encounter.iloc[i]["ID"], "TS_start": max(time_starts), "TS_end": min(time_ends), "Type": enc_type, "Edge_ID": edge_id}
                df_edges.loc[len(df_edges)] = new_row_2
    return df_edges

def update_states(df_nodes, df_edges, edge_id, timestamp_start, timestamp_end, encounter_type):
    global df_edges_last_meeting, agent_states, new_row_states, df_states_after, encounters, sick_exp_people 
    df_edges_last_meeting = df_edges.loc[df_edges["Edge_ID"]==edge_id]
    agents_at_meeting = df_edges_last_meeting["ID_1"].unique()
    agent_states = pd.DataFrame(columns = ["ID", "Last_state"])
    for agent_id in agents_at_meeting:
        agent_id_states = df_nodes.loc[df_nodes["ID"] == agent_id]
        agent_last_state = agent_id_states.loc[agent_id_states["Datetime"] == max(agent_id_states[agent_id_states["Datetime"]<=timestamp_start]["Datetime"]), "State"].values[0]
        new_row_agent_states = {"ID": agent_id, "Last_state": agent_last_state}
        agent_states.loc[len(agent_states)] = new_row_agent_states
    for i in range(len(agent_states)):
        df_states_after = df_nodes.loc[df_nodes["ID"]==agent_states.iloc[i]["ID"]]
        df_states_after = df_states_after.loc[df_states_after["Datetime"] >= timestamp_start]
        if len(df_states_after) == 0:
            if agent_states.iloc[i]["Last_state"] == "healthy" or agent_states.iloc[i]["Last_state"] == "recovered":
                encounters = df_edges.loc[df_edges["ID_1"]==agent_states.iloc[i]["ID"]]
                encounters = encounters.loc[encounters["Edge_ID"] == edge_id]
                sick_exp_people = pd.DataFrame([p for k, p in encounters.iterrows() if agent_states.loc[agent_states["ID"] == p["ID_2"], "Last_state"].values[0] == "sick" or agent_states.loc[agent_states["ID"] == p["ID_2"], "Last_state"].values[0] == "exposed"])
                if len(sick_exp_people) != 0:
                    total_exposure_time_secs = 0
                    for ii, row in sick_exp_people.iterrows():
                        total_exposure_time_secs += (row["TS_end"] - row["TS_start"]).total_seconds()
                    if agent_states.iloc[i]["Last_state"] == "healthy":
                        prob_h_to_exposed = healthy_to_exp(agent_states.iloc[i]["ID"], total_exposure_time_secs, encounter_type)
                        new_state = random.choices(["healthy", "exposed"], weights = (1-prob_h_to_exposed, prob_h_to_exposed), k = 1)[0]
                    else:
                        prob_r_to_exposed = recovered_to_exp(agent_states.iloc[i]["ID"], total_exposure_time_secs, encounter_type)
                        new_state = random.choices(["recovered", "exposed"], weights = (1-prob_r_to_exposed, prob_r_to_exposed), k = 1)[0]
                    new_row_state = {"ID": agent_states.iloc[i]["ID"], "State": new_state, "Datetime": timestamp_end}
                    df_nodes.loc[len(df_nodes)] = new_row_state
                    if new_state == "exposed":
                        nr_days_exposed = min(max(1,np.round(np.random.normal(7, 2, 1))), 1)
                        sick_day = timestamp_end + datetime.timedelta(days = int(nr_days_exposed))
                        new_row_state = {"ID": agent_states.iloc[i]["ID"], "State": "sick", "Datetime": sick_day} # CAMBIALO
                        df_nodes.loc[len(df_nodes)] = new_row_state
                        p_death = 0.01
                        is_dead = random.choices(["dead", "recovered"], weights = (p_death, 1-p_death), k = 1)[0]
                        if is_dead == "dead":
                            nr_days_new_state = min(max(1,np.round(np.random.normal(3, 1, 1))), 1)
                        else:
                            nr_days_new_state = min(max(1,np.round(np.random.normal(10, 3, 1))), 1)
                        new_state_day = sick_day + datetime.timedelta(days = int(nr_days_new_state))
                        new_row_state = {"ID": agent_states.iloc[i]["ID"], "State": is_dead, "Datetime": new_state_day}
                        df_nodes.loc[len(df_nodes)] = new_row_state
                else:
                    new_row_state = {"ID": agent_states.iloc[i]["ID"], "State": agent_states.iloc[i]["Last_state"], "Datetime": timestamp_end}
                    df_nodes.loc[len(df_nodes)] = new_row_state
                    
    return df_nodes
                        
                    # Abbiamo calcolato l'esposizione alle persone malate, ora bisogna calcolare la probabilità che diventi
                    # esposto e, se lo diventa, aggiungiamo anche una riga in nodes che definisce quando diventerà malato.
                    # Il discorso del morto lo aggiungiamo dopo nel malato
                    # Poi restano da definire gli altri stati: se è esposto non fa nulla e anche se è malato e se uno è recovered
                    # la probabilità di tornare malato cambia
                
        
            
        
    

In [5]:
# Distribuzioni di probabilità

def encounters_number_dist(N, enc_type):
    if enc_type == "kiss":
        return 2
    elif enc_type == "wave":
        return 2
    elif enc_type == "talk":
        return min(max(2,round(N/5)), 10)
    elif enc_type == "party":
        return min(max(2,round(N/1)), 50)
    elif enc_type == "lunch":
        return min(max(2,round(N/2)), 10) 
    elif enc_type == "dinner":
        return min(max(2,round(N/2)), 10)
    elif enc_type == "lecture":
        return min(max(2,round(N/2)), 30)
    elif enc_type == "tango_lesson":
        return min(max(2,round(N/3)), 20)
    
def duration_dist(enc_type):
    """Given the encounter type returns the duration in seconds"""
    if enc_type == "kiss":
        return min(max(10, np.round(np.random.normal(15, 1, 1))), 120)
    elif enc_type == "wave":
        return min(max(30, np.round(np.random.normal(45, 1, 1))), 60)
    elif enc_type == "talk":
        return min(max(60, np.round(np.random.normal(600, 50, 1))), 2400)
    elif enc_type == "party":
        return min(max(3600, np.round(np.random.normal(3*3600, 3600, 1))), 6*3600)
    elif enc_type == "lunch":
        return min(max(600, np.round(np.random.normal(3600, 1800, 1))), 3600*3)
    elif enc_type == "dinner":
        return min(max(600, np.round(np.random.normal(3600, 1800, 1))), 3600*3)
    elif enc_type == "lecture":
        return min(max(3600, np.round(np.random.normal(3600*2, 3600, 1))), 3600*6)
    elif enc_type == "tango_lesson":
        return min(max(1800, np.round(np.random.normal(3600, 600, 1))), 3600*2)
    
def delay_timestamp_start_end(enc_type, timestamp_start, timestamp_end):
    if enc_type == "kiss":
        return timestamp_start, timestamp_end
    elif enc_type == "wave":
        return timestamp_start, timestamp_end
    elif enc_type == "talk":
        early_late = random.choice(["early", "late"])
        if early_late == "early":
            return timestamp_start, timestamp_end
        else:
            timestamp_start_new = timestamp_start + datetime.timedelta(seconds = int(np.random.uniform(0, 300)))
            timestamp_end_new = timestamp_end - datetime.timedelta(seconds = int(np.random.uniform(0, 300)))
            return timestamp_start_new, timestamp_end_new
    elif enc_type == "party":
        early_late = random.choice(["early", "late"])
        if early_late == "early":
            return timestamp_start, timestamp_end
        else:
            timestamp_start_new = timestamp_start + datetime.timedelta(seconds = int(np.random.uniform(0, 3600)))
            timestamp_end_new = timestamp_end - datetime.timedelta(seconds = int(np.random.uniform(0, 5400)))
            return timestamp_start_new, timestamp_end_new
    elif enc_type == "lunch":
        early_late = random.choice(["early", "late"])
        if early_late == "early":
            return timestamp_start, timestamp_end
        else:
            timestamp_start_new = timestamp_start + datetime.timedelta(seconds = int(np.random.uniform(0, 600)))
            timestamp_end_new = timestamp_end - datetime.timedelta(seconds = int(np.random.uniform(0, 1800)))
            return timestamp_start_new, timestamp_end_new
    elif enc_type == "dinner":
        early_late = random.choice(["early", "late"])
        if early_late == "early":
            return timestamp_start, timestamp_end
        else:
            timestamp_start_new = timestamp_start + datetime.timedelta(seconds = int(np.random.uniform(0, 600)))
            timestamp_end_new = timestamp_end - datetime.timedelta(seconds = int(np.random.uniform(0, 1800)))
            return timestamp_start_new, timestamp_end_new
    elif enc_type == "lecture":
        early_late = random.choice(["early", "late"])
        if early_late == "early":
            return timestamp_start, timestamp_end
        else:
            timestamp_start_new = timestamp_start + datetime.timedelta(seconds = int(np.random.uniform(0, 3600)))
            timestamp_end_new = timestamp_end - datetime.timedelta(seconds = int(np.random.uniform(0, 600)))
            return timestamp_start_new, timestamp_end_new
    elif enc_type == "tango_lesson":
        early_late = random.choice(["early", "late"])
        if early_late == "early":
            return timestamp_start, timestamp_end
        else:
            timestamp_start_new = timestamp_start + datetime.timedelta(seconds = int(np.random.uniform(0, 300)))
            timestamp_end_new = timestamp_end - datetime.timedelta(seconds = int(np.random.uniform(0, 300)))
            return timestamp_start_new, timestamp_end_new
    
def healthy_to_exp(agent_id, total_exposure_time, enc_type):
    age = df_properties.loc[df_properties["ID"]==agent_id, "Age"]
    if enc_type == "kiss":
        return 0.5 + 0.4*total_exposure_time/120
    elif enc_type == "wave":
        return 0.1 + 0.2*total_exposure_time/60
    elif enc_type == "talk":
        return 0.4 + 0.3*total_exposure_time/2400
    elif enc_type == "party":
        return 0.6 + 0.4*total_exposure_time/(6*3600)
    elif enc_type == "lunch":
        return 0.3 + 0.2*total_exposure_time/(3600*3)
    elif enc_type == "dinner":
        return 0.3 + 0.2*total_exposure_time/(3600*3)
    elif enc_type == "lecture":
        return 0.4 + 0.4*total_exposure_time/(3600*6)
    elif enc_type == "tango_lesson":
        return 0.5 + 0.4*total_exposure_time/(3600*2)
    
def recovered_to_exp(agent_id, total_exposure_time, enc_type):
    age = df_properties.loc[df_properties["ID"]==agent_id, "Age"]
    if enc_type == "kiss":
        return 0.4 + 0.4*total_exposure_time/120
    elif enc_type == "wave":
        return 0.0 + 0.2*total_exposure_time/60
    elif enc_type == "talk":
        return 0.2 + 0.3*total_exposure_time/2400
    elif enc_type == "party":
        return 0.5 + 0.4*total_exposure_time/(6*3600)
    elif enc_type == "lunch":
        return 0.2 + 0.2*total_exposure_time/(3600*3)
    elif enc_type == "dinner":
        return 0.2 + 0.2*total_exposure_time/(3600*3)
    elif enc_type == "lecture":
        return 0.3 + 0.4*total_exposure_time/(3600*6)
    elif enc_type == "tango_lesson":
        return 0.3 + 0.4*total_exposure_time/(3600*2)
     
        
    
        


In [6]:
class MoneyAgent(mesa.Agent):
    def __init__(self, unique_id, model, sex, age, state):
        super().__init__(unique_id, model)
        self.sex = sex
        self.age = age
        self.state = state
        
        # self.counter = 0 if self.state == "healthy" else 7
        
    def step(self):
        global edge_id, df_nodes, df_edges, df_states_self, timestamp_start_ref, self_last_state_date, timestamp_end, duration, df_times_encounter, df_states_agent, encounter_type, timestamp_start_enc, timestamp_end_enc, num_agents_encountered, i
        edge_id += 1
        encounter_type = random.choices(list(encounters_prob.keys()), list(encounters_prob.values()), k = 1)[0]
        num_agents_encountered = encounters_number_dist(num_agents, encounter_type)
        timestamp_start_ref = max(df_edges.loc[df_edges["ID_1"]==self.unique_id, "TS_end"]) + datetime.timedelta(hours=1)
        df_states_self = df_nodes.loc[df_nodes["ID"] == self.unique_id, ["State", "Datetime"]]
        if str(type(df_states_self)) == "<class 'pandas.core.series.Series'>":
            df_states_self = pd.DataFrame([df_states_self.values], columns = df_states_self.index)
        self_last_state_date = df_states_self.loc[df_states_self["Datetime"] == max(df_states_self[df_states_self["Datetime"]<=timestamp_start_ref]["Datetime"])][["State", "Datetime"]]
        if self_last_state_date["State"].values[0] != "dead": 
            duration = duration_dist(encounter_type)
            timestamp_end = timestamp_start_ref + datetime.timedelta(seconds = int(duration))
            df_states_self = df_nodes.loc[df_nodes["ID"] == self.unique_id, ["State", "Datetime"]]
            if str(type(df_states_self)) == "<class 'pandas.core.series.Series'>":
                df_states_self = pd.DataFrame([df_states_self.values], columns = df_states_self.index)
            self_last_state_date = df_states_self.loc[df_states_self["Datetime"] == max(df_states_self[df_states_self["Datetime"]<=timestamp_end]["Datetime"])][["State", "Datetime"]]
            if self_last_state_date["State"].values[0] != "dead":
                # Verifica se è possibile l'incontro per self 
                if check_times(df_edges, self.unique_id, timestamp_start_ref, timestamp_end):
                    df_times_encounter = pd.DataFrame(columns = ["ID", "TS_start", "TS_end"])
                    new_row_edge_id = {"ID": self.unique_id, "TS_start": timestamp_start_ref, "TS_end": timestamp_end}
                    df_times_encounter.loc[len(df_times_encounter)] = new_row_edge_id
                    agents_encountered = [] # lista degli agenti incontrati (pesco da agents_list)
                    i = 0
                    agents_list_2 = []
                    # Prendo gli id degli agenti
                    agents_list = [a for a in self.model.schedule.agents if a!=self] # lista degli agenti pescabili
                    for ag in agents_list:
                        df_states_agent = df_nodes[df_nodes["ID"]==ag.unique_id][["State", "Datetime"]]
                        agent_last_state_2_before = df_states_agent[df_states_agent["Datetime"] == max(df_states_agent[df_states_agent["Datetime"]<=timestamp_start_ref]["Datetime"])]["State"].values[0]
                        agent_last_state_2_after = df_states_agent[df_states_agent["Datetime"] == max(df_states_agent[df_states_agent["Datetime"]<=timestamp_end]["Datetime"])]["State"].values[0]
                        if agent_last_state_2_before != "dead" and agent_last_state_2_after != "dead":
                            agents_list_2.append(ag)
                    # se è possibile l'incontro per self allora pesca a caso gli agenti
                    while i<num_agents_encountered-1 and len(agents_list_2)!=0:
                        other_agent = self.random.choice(agents_list_2)
                        agents_list_2.remove(other_agent)
                        agents_encountered.append(other_agent)
                        timestamp_start_enc, timestamp_end_enc = delay_timestamp_start_end(encounter_type, timestamp_start_ref, timestamp_end)
                        # verifica se è possibile l'incontro per entrambi gli agenti guardando se è compredo tra due incontri già presenti
                        if check_times(df_edges, other_agent.unique_id, timestamp_start_enc, timestamp_end_enc):
                            # Popoliamo il dataframe relativo all'incontro
                            new_row_edge_id = {"ID": other_agent.unique_id, "TS_start": timestamp_start_enc, "TS_end": timestamp_end_enc} 
                            df_times_encounter.loc[len(df_times_encounter)] = new_row_edge_id
                            i += 1    
                    if i>=num_agents_encountered/2: # Se il numero di agenti incontrati è soddisfacente allora genera incontro
                        df_edges = find_overlaps(df_times_encounter, df_edges, edge_id, encounter_type)
                        # Qui l'incontro è stato generato, quindi sistemiamo gli stati
                        df_nodes = update_states(df_nodes, df_edges, edge_id, timestamp_start_ref, timestamp_end, encounter_type)
                        # Sistemati gli stati

                    # Mancano tutte le distribuzioni di probabilità per nr di persone incontrate, nr. incontri, timestamps deviati
                    # rispetto a quelli principali e probabilità dei passaggi di stato
                    # Manca soprattutto modellare i passaggi di stato
                    # Ricordati che per ogni agente il timestamp iniziale (da quando devi generare gli incontri) corrisponde
                    # al suo ultimo passaggio di stato (da salvare nel dataframe nodes)
                    # AGGIORNA EDGEID
                    # Crea il dataframe vuoto df_edges
                    # Crea il dataframe con le proprietà degli agenti

                
                  
class MoneyModel(mesa.Model):
    def __init__(self, N):
        global df_properties, df_nodes, df_edges
        self.num_agents = N
        self.schedule = mesa.time.RandomActivation(self)
        ages = random.choices(range(10, 80), k=self.num_agents)
        diseases = random.choices(["sick", "healthy"], k=self.num_agents)
        sexs = random.choices(["female", "male"], k=self.num_agents)
        # Creating agents
        for i in range(self.num_agents):
            a = MoneyAgent(i, self, sexs[i], ages[i], diseases[i])
            self.schedule.add(a)
            new_row_df_properties = {"ID": i, "Sex": a.sex, "Age": a.age}
            df_properties = df_properties.append(new_row_df_properties, ignore_index = True)
            new_row_df_nodes = {"ID": i, "State": a.state, "Datetime": datetime.datetime(2020, 1, 1, 8, 0, 0)}
            df_nodes = df_nodes.append(new_row_df_nodes, ignore_index = True)
            new_row_df_edges = {"ID_1": i, "ID_2": i, "TS_start": datetime.datetime(2020, 1, 1, 8, 0, 0), "TS_end": datetime.datetime(2020, 1, 1, 8, 0, 0), "Type": "INIT", "Edge_ID": "INIT"}
            df_edges = df_edges.append(new_row_df_edges, ignore_index = True)
            if a.state == "sick":
                nr_days_recovering = min(max(7,np.round(np.random.normal(10, 3, 1))), 14)
                new_state_day = datetime.datetime(2020, 1, 1, 8, 0, 0) + datetime.timedelta(days = int(nr_days_recovering))
                new_row_df_nodes = {"ID": i, "State": "recovered", "Datetime": new_state_day}
                df_nodes = df_nodes.append(new_row_df_nodes, ignore_index = True)
                
    def step(self):
        self.schedule.step()
    
    

In [7]:
steps = 100
empty_model = MoneyModel(num_agents)
for i in range(steps):
    empty_model.step()
        

/tmp/ipykernel_48/2727201778.py:86: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_properties = df_properties.append(new_row_df_properties, ignore_index = True)
/tmp/ipykernel_48/2727201778.py:88: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_nodes = df_nodes.append(new_row_df_nodes, ignore_index = True)
/tmp/ipykernel_48/2727201778.py:90: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_edges = df_edges.append(new_row_df_edges, ignore_index = True)
/tmp/ipykernel_48/2727201778.py:95: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_nodes = df_nodes.append(new_row_df_nodes, ignore_index = True)
/tmp/ipykernel_48/2727201778.py:86: F

In [8]:
df_nodes[df_nodes["State"]=="dead"]

,ID,State,Datetime
228,7,dead,2020-01-19 12:28:10
476,8,dead,2020-02-05 00:38:58


In [9]:
df_edges

,ID_1,ID_2,TS_start,TS_end,Type,Edge_ID
0,0,0,2020-01-01 08:00:00,2020-01-01 08:00:00,INIT,INIT
1,1,1,2020-01-01 08:00:00,2020-01-01 08:00:00,INIT,INIT
2,2,2,2020-01-01 08:00:00,2020-01-01 08:00:00,INIT,INIT
3,3,3,2020-01-01 08:00:00,2020-01-01 08:00:00,INIT,INIT
4,4,4,2020-01-01 08:00:00,2020-01-01 08:00:00,INIT,INIT
...,...,...,...,...,...,...
9557,5,0,2020-02-05 04:18:37,2020-02-05 04:27:33,talk,998
9558,3,6,2020-02-05 06:57:48,2020-02-05 07:00:41,talk,999
9559,6,3,2020-02-05 06:57:48,2020-02-05 07:00:41,talk,999
9560,9,5,2020-02-05 07:59:15,2020-02-05 07:59:59,wave,1000


In [10]:
df_edges

,ID_1,ID_2,TS_start,TS_end,Type,Edge_ID
0,0,0,2020-01-01 08:00:00,2020-01-01 08:00:00,INIT,INIT
1,1,1,2020-01-01 08:00:00,2020-01-01 08:00:00,INIT,INIT
2,2,2,2020-01-01 08:00:00,2020-01-01 08:00:00,INIT,INIT
3,3,3,2020-01-01 08:00:00,2020-01-01 08:00:00,INIT,INIT
4,4,4,2020-01-01 08:00:00,2020-01-01 08:00:00,INIT,INIT
...,...,...,...,...,...,...
9557,5,0,2020-02-05 04:18:37,2020-02-05 04:27:33,talk,998
9558,3,6,2020-02-05 06:57:48,2020-02-05 07:00:41,talk,999
9559,6,3,2020-02-05 06:57:48,2020-02-05 07:00:41,talk,999
9560,9,5,2020-02-05 07:59:15,2020-02-05 07:59:59,wave,1000


In [90]:
self_last_state_date["State"].values[0]

'exposed'

In [91]:
self_last_state_date["State"].values[0] != "dead"

True

In [87]:
timestamp_start_ref

Timestamp('2020-02-03 01:13:41')

In [11]:
df_edges.to_csv("tNodeEmbed/data/Meetings/edges_1_prova.csv",columns = list(df_edges.columns), index = False)

In [12]:
df_nodes.to_csv("tNodeEmbed/data/Meetings/nodes_1_prova.csv",columns = list(df_nodes.columns), index = False)